# 1. Necessary Headfiles

In [14]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc

# 2. Prepare Datasets

## 2.1. Navigate to the data folder 

In [15]:
root_dir = "../eegdata"
# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "conditionA"
# condition_dir: hc, mcs, uws
condi_dir = "hc"

In [16]:
# distribution index matrix
matrix_index = np.array([
    [0,0,2,2,4,4,6,6,8,8,2,2,4,4,6,6,2,4,6,3,3,5,5,3,3,5,5,3,3,2,2,4,4,6,6,1,1,3,3,5,5,7,7,2,2,4,4,6,6,1,1,3,3,5,5,7,7,0,5],
    [4,6,3,7,3,7,3,7,4,6,1,9,1,9,1,9,5,5,5,4,6,4,6,2,8,2,8,0,10,4,6,4,6,4,6,4,6,3,7,3,7,4,6,2,8,2,8,2,8,3,7,1,9,1,9,3,7,5,5]
])

## 2.2. Load the data

In [ ]:
# 设置使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = MyData(root_dir,exper_dir,condi_dir) # eg.conditionA\hc

data_map_all = None # eg.conditionA\hc下所有被试map集合
for person in tqdm(range(len(dataset))): #12种情况中的1种，其中的所有被试
    # filepath是最子文件夹中每个.mat文件的名字
    # path是包含当前情况的.mat文件的子文件夹
    filename = os.path.join(dataset.path,dataset.file_path[person]) # eg. conditionA\hc\hc1.set.mat
    data = scipy.io.loadmat(filename)# 读取该被试的数据为字典
    data = data['datas']# 键值对读取59*x的数据矩阵
    
    # 将数据移动到GPU上
    data = torch.from_numpy(data).to(device)
    
    # 在GPU上处理数据
    # 创建被试全0map
    data_map_person = None 
    # 遍历该被试的所有段
    for para in tqdm(range(data.shape[2])): 
        # 创建段全0map
        data_map_para = None 
        for point in range(data.shape[1]): # 遍历该段所有数据点
            # 创建单个数据点全0map
            data_map_point = torch.zeros((10, 11)).to(device) 
            # 遍历59电极数据并赋值给全0map
            for channel in range(data.shape[0]): 
                data_map_point[matrix_index[0][channel]][matrix_index[1][channel]] = data[channel][point][para]
            # 保存单个数据点map到段map
            if data_map_para is None:
                data_map_para = data_map_point.unsqueeze(0)
            else:
                data_map_para = torch.cat((data_map_para, data_map_point.unsqueeze(0)), dim=0)
        # 保存一段map到被试map
        if data_map_person is None:
            data_map_person = data_map_para.unsqueeze(0)
        else:
            data_map_person = torch.cat((data_map_person, data_map_para.unsqueeze(0)), dim=0)
    # 保存一个被试map
    if data_map_all is None:
        data_map_all = data_map_person.unsqueeze(0)
    else:
        data_map_all = torch.cat((data_map_all, data_map_person), dim=0) # 修改此处
    
    # 遍历一个被试，清理一次gpu内存
    gc.collect()
    torch.cuda.empty_cache()


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/20 [00:00<?, ?it/s]

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
print(data_map_all.shape)
print(data.shape)
print(len(dataset))

## 2.3 Save the Data

In [7]:
torch.save(data_map_all,'..\eegmap_direct\conditionA\A_hc.pt')

In [155]:
import numpy as np

mat_a = np.array([
    [1,2,3,4],
    [5,6,7,8]
])
mat_index = np.array([
    [0,0,1,1],
    [0,1,0,1]
])

keep_map = None

for j in range(mat_a.shape[0]):
    map = np.zeros((2,2))
    for i in range(mat_a.shape[1]):
        map[mat_index[0][i]][mat_index[1][i]] = mat_a[j][i]
    if keep_map is None:
        keep_map = map[np.newaxis, ...]
    else:
        keep_map = np.concatenate((keep_map, map[np.newaxis, ...]), axis=0)

print(keep_map)


[[[1. 2.]
  [3. 4.]]

 [[5. 6.]
  [7. 8.]]]


In [157]:
print(keep_map[1])


[[5. 6.]
 [7. 8.]]
